In [101]:
# imports
import pandas as pd
import matplotlib
import math
import random
import numpy
import geopandas as gpd
from shapely.ops import unary_union
from shapely.geometry import Point

%matplotlib inline

# avoiding Type 3 fonts in matplotlib plots
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

font = {'size'   : 20}

matplotlib.rc('font', **font)
matplotlib.rc('lines', linewidth=2.0)
matplotlib.rc('lines', markersize=8)

numpy.set_printoptions(threshold=numpy.inf)

pd.set_option('display.max_rows', 500)


# transform polygon from epsg:4326 to epsg:3035 to work in km
def transform_polygon(epsg_from, epsg_to, polygon):
    # create a GeoDataFrame
    gdf = gpd.GeoDataFrame(index=[0], crs=epsg_from, geometry=[polygon])

    # transform the GeoDataFrame to EPSG:3035
    return gdf.to_crs(epsg_to)['geometry'][0]

In [102]:
# configure simulation
input_file = 'data/gis_incidences_nona.pkl'
gis_incidences_nona=pd.read_pickle(input_file)
number_of_simulations = 1000000 # equivalent to number of points

lk_identifier = (222, 223) # SK Wolfsburg and LK Helmstedt

# print(gis_incidences_nona)

for lk in lk_identifier:
    print(gis_incidences_nona['LK'][lk])
    
array_of_lks = []
for lk in lk_identifier:
    array_of_lks.append(gis_incidences_nona['geometry'][lk])
merged_lks = unary_union(array_of_lks)
merged_lks_km = transform_polygon('epsg:4326', 'epsg:3035', merged_lks)

# print(merged_lks_km)

for lk in lk_identifier:
    print("Bounds of lk '" + gis_incidences_nona['LK'][lk] + "': " + str(transform_polygon('epsg:4326', 'epsg:3035', gis_incidences_nona['geometry'][lk]).bounds))

print("Bounds of merged lks in km: " + str(merged_lks_km.bounds))

SK Wolfsburg
LK Helmstedt
Bounds of lk 'SK Wolfsburg': (4365251.516081874, 3245533.407448974, 4382696.453068134, 3265422.7453582273)
Bounds of lk 'LK Helmstedt': (4362094.160649367, 3216018.6549146227, 4395594.045104617, 3262720.0948905647)
Bounds of merged lks in km: (4362094.160649367, 3216018.6549146227, 4395594.045104617, 3265422.7453582273)


In [103]:
# generate points in the given lk's by using their merged boundaries
def random_point_in_bounds_of_polygon(polygon):
    minx, miny, maxx, maxy = polygon.bounds
    x = numpy.random.uniform( minx, maxx )
    y = numpy.random.uniform( miny, maxy )
    return Point(x, y)


i = 0
points_in_lks = []
while i <= number_of_simulations:
    point = random_point_in_bounds_of_polygon(merged_lks_km)
    if merged_lks_km.contains(point):
        points_in_lks.append(point)
        i += 1

In [109]:
# simulate movement and count the number of movements, which end in the same lks
a = 2.13
number_of_movements_inside_lks = 0
i = 0
for point in points_in_lks:
    angle = random.uniform(0, 2 * math.pi)
    x = random.uniform(0, 1)
    d = (math.pow(x, (-1 / (a - 1))))
    pos_x = (point.x / 1000.0) + (d * math.cos(angle)) # we divide by 1000, since epsg:3035 works with meters
    pos_y = (point.y / 1000.0) + (d * math.sin(angle))

    if False: #i % 10000 == 0:
        print("pos x: " + str(pos_x) + "; added distance: " + str(d * math.cos(angle)))
        print("pos y: " + str(pos_y) + "; added distance: " + str(d * math.sin(angle)))

    if merged_lks_km.contains(Point(pos_x * 1000.0, pos_y * 1000.0)): # to check if our new point is inside our polygon, we multiply by 1000 since epsg:3035 works with meters
        number_of_movements_inside_lks += 1

    i += 1

print("Number of movements that ended inside the same LKs: " + str(number_of_movements_inside_lks))
print("Percentage: " + str((number_of_movements_inside_lks / number_of_simulations) * 100)  + "%")

Number of movements that ended inside the same LKs: 820768
Percentage: 82.0768%
